In [1]:
import os

In [2]:
%pwd

'/media/kalema/9954-79C8/Projects/Swahili-News-Classifier/research'

In [3]:
os.chdir('../')


In [4]:
%pwd

'/media/kalema/9954-79C8/Projects/Swahili-News-Classifier'

In [7]:
from dataclasses import dataclass
from pathlib import Path

from swahiliNewsClassifier.constants import *
from swahiliNewsClassifier.utils.helper_functions import read_yaml, create_directories

ModuleNotFoundError: No module named 'swahiliNewsClassifier'